In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  8 15:35:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic InRelease
Readin

In [3]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ui66g42t
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ui66g42t
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import hopsworks
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) liangc40
	 (2) Lab1_for_iris

Enter project to access: 1

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5311


In [6]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="chinese", task="transcribe")

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="chinese", task="transcribe")

In [7]:
# TODO - download the dataset from Hopsworks
! rm -rf common_voice
import os
from datasets import list_datasets
os.mkdir("common_voice")
os.mkdir("common_voice/train")
os.mkdir("common_voice/test")

dataset_api = project.get_dataset_api()

downloaded = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/train/dataset.arrow", local_path="./common_voice/train/")
downloaded = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/test/dataset.arrow", local_path="./common_voice/test/")
downloaded = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/train/dataset_info.json", local_path="./common_voice/train/")
downloaded = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/test/dataset_info.json", local_path="./common_voice/test/")
downloaded_file_path = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/train/state.json", local_path="./common_voice/train/")

downloaded_file_path = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/test/state.json", local_path="./common_voice/test/")

downloaded_file_path = dataset_api.download(
    "liangc40_Training_Datasets/common_voice_cn/dataset_dict.json", local_path="./common_voice/")

Downloading: 0.000%|          | 0/1410516408 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/314183344 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1143 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1143 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/289 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/289 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/29 elapsed<00:00 remaining<?

In [23]:
from datasets import  DatasetDict
common_voice = DatasetDict.load_from_disk("common_voice")

In [24]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [25]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [26]:
import evaluate

metric = evaluate.load("wer")

In [27]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [28]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai--whisper-small/snapshots/ecd562088fc463cdf07bf4a997944edeae03993e/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-small",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448

In [45]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [80]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="./whisper-small-hi_02",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=50,
    max_steps=800,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

PyTorch: setting up devices


In [81]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/content/./whisper-small-hi_02 is already a clone of https://huggingface.co/liangc40/whisper-small-hi_02. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [82]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./whisper-small-hi_02/preprocessor_config.json
tokenizer config file saved in ./whisper-small-hi_02/tokenizer_config.json
Special tokens file saved in ./whisper-small-hi_02/special_tokens_map.json
added tokens file saved in ./whisper-small-hi_02/added_tokens.json


In [87]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

trainer.train()

***** Running training *****
  Num examples = 1468
  Num Epochs = 9
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 800
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.051200,0.767606,70.163542
200,0.033900,0.721468,68.887116
300,0.012900,0.766090,66.573594
400,0.004800,0.770185,66.055046
500,0.003700,0.778455,64.658955
600,0.000600,0.808508,63.262864
700,0.000700,0.829735,63.103311
800,0.000100,0.838156,62.983646


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

TrainOutput(global_step=800, training_loss=0.014937262861931231, metrics={'train_runtime': 6302.8351, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.127, 'total_flos': 3.68465839128576e+18, 'train_loss': 0.014937262861931231, 'epoch': 8.7})

In [90]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "sv",
    "model_name": "my_tuned_whisper_cn",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [91]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to ./whisper-small-hi_02
Configuration saved in ./whisper-small-hi_02/config.json
Model weights saved in ./whisper-small-hi_02/pytorch_model.bin
Feature extractor saved in ./whisper-small-hi_02/preprocessor_config.json
